In [5]:
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler

# Elaboracion de archivos para la API

Se crearon los DataFrames con la informacion justa para la implementacion de cada funcion requerida. Para cada funcion se determinaron variables criticas (ej. el parametro de entrada) que de ser nulas el registro correspondiente no tendria la iformacion minima para ser incluido en el analisis. De acuerdo con esto, se decidio eliminar estos registros.

## Cargar tablas

In [ ]:
games=pd.read_parquet("Archivos EDA/games_eda.parquet")

In [ ]:
items=pd.read_parquet("Archivos EDA/items_eda.parquet")

In [ ]:
reviews=pd.read_parquet("Archivos EDA/reviews_eda.parquet")

In [ ]:
games_def

In [ ]:
items

In [ ]:
reviews

## Creacion de archivos

### def developer( desarrollador : str )

Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. <br><br>
Ejemplo de retorno: <br>
| Año   | Cantidad de items | Contenido Free |
| ----- | ----------------- | -------------- |
| 2023  | 50                | 27%            |
| 2022  | 45                | 25%            |
<br>

Columnas necesarias:
- developer (games)
- release_year (games)
- price (games)
- item_id (games)

In [ ]:
def_developer = games[['developer','item_id','release_year','price']]
def_developer.drop(def_developer[def_developer['developer'].isnull()].index,inplace=True)
def_developer

In [ ]:
def_developer.isnull().sum()

### def userdata( User_id : str )


Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items. <br><br>
Ejemplo de retorno: <br>
{"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}
<br><br>
Columnas necesarias:
- user_id (reviews)
- recommend (reviews)
- item_id (reviews, games)
- price (games)

In [ ]:
def_userdata = reviews[['user_id','item_id','recommend']]
def_userdata = def_userdata.merge(games[['item_id','price']],on='item_id')
def_userdata

In [ ]:
def_userdata.isnull().sum()

### def UserForGenre( genero : str )

Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento. <br><br>
Ejemplo de retorno:<br>
{"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}
<br><br>
Columnas necesarias:
- user_id (items)
- playtime (items)
- item_id (items, games)
- genres (games)
- release_year (games)

In [ ]:
def_userforgenre = items[['user_id','playtime','item_id']]
def_userforgenre = def_userforgenre.merge(games[['item_id','genres','release_year']],on='item_id')
def_userforgenre.drop(def_userforgenre[def_userforgenre['genres'].isnull()].index,inplace=True)
def_userforgenre

In [ ]:
def_userforgenre.isnull().sum()

### def best_developer_year( año : int )

Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos) <br><br>
Ejemplo de retorno: <br>
[{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]
<br><br>
Columnas necesarias:
- developer (games)
- release_year (games)
- recommend (reviews)
- sentiment_analysis (reviews)
- item_id (reviews, games)

In [ ]:
def_best_dev = games[['developer','item_id','release_year']]
def_best_dev = def_best_dev.merge(reviews[['item_id','recommend','sentiment_analysis']],on='item_id')
def_best_dev.drop(def_best_dev[(def_best_dev['developer'].isnull())|(def_best_dev['release_year'].isnull())].index,inplace=True)
def_best_dev

In [ ]:
def_best_dev.isnull().sum()

### def developer_reviews_analysis( desarrolladora : str )

Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo. <br><br>
Ejemplo de retorno: <br>
{'Valve' : [Negative = 182, Positive = 278]}
<br><br>
Columnas necesarias:
- developer (games)
- item_id (games, reviews)
- sentiment_analysis (reviews)

In [ ]:
def_dev_rev = games[['developer','item_id']]
def_dev_rev = def_dev_rev.merge(reviews[['item_id','sentiment_analysis']],on='item_id')
def_dev_rev.drop(def_dev_rev[def_dev_rev['developer'].isnull()].index,inplace=True)
def_dev_rev

In [ ]:
def_dev_rev.isnull().sum()

### def recomendacion_juego (id_producto : int)

In [ ]:
def_recom = games.copy().reset_index()
def_recom['features'] = def_recom.apply(lambda row: 
                              (list(row['genres']) if row['genres'] is not None else [])+
                              (list(row['tags']) if row['tags'] is not None else []) + 
                              (list(row['specs']) if row['specs'] is not None else []), 
                              axis=1)
def_recom = def_recom[['item_id','item_name','price','early_access','release_year','features']]
scaler = StandardScaler()
def_recom['price'] = scaler.fit_transform(def_recom[['price']])
def_recom['release_year'] = scaler.fit_transform(def_recom[['release_year']])

mlb = MultiLabelBinarizer()
feat_dummies = pd.DataFrame(mlb.fit_transform(def_recom['features']), columns=mlb.classes_, index=def_recom.index)
def_recom = pd.concat([def_recom, feat_dummies], axis=1)

def_recom.drop(columns='features',inplace=True)
def_recom.fillna(0,inplace=True)

## Exportar archivos finales

In [ ]:
pq.write_table(pa.Table.from_pandas(def_developer),'Archivos API/def_developer.parquet')
pq.write_table(pa.Table.from_pandas(def_userdata),'Archivos API/def_userdata.parquet')
pq.write_table(pa.Table.from_pandas(def_userforgenre),'Archivos API/def_userforgenre.parquet')
pq.write_table(pa.Table.from_pandas(def_best_dev),'Archivos API/def_best_dev.parquet')
pq.write_table(pa.Table.from_pandas(def_dev_rev),'Archivos API/def_dev_rev.parquet')
pq.write_table(pa.Table.from_pandas(def_recom),'Archivos API/def_recom.parquet')